# Bitcoin Trading Strategy Backtesting Examples

This notebook demonstrates how to use the backtesting framework to evaluate trading strategies on historical Bitcoin price data. We'll cover:

1. Basic strategy backtesting
2. Strategy parameter optimization
3. Walk-forward analysis
4. Visualization of results
5. Comparing multiple strategies

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests

# Add the project root to the path so we can import our modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our backtesting modules
from app.trading.backtester import Backtester
from app.trading.trading_strategies import Strategy, Action
from app.trading.indicators import add_all_indicators
from app.trading.trading_visualization import plot_equity_curve, plot_returns_distribution, plot_underwater_curve
from app.trading.trading_utils import calculate_sharpe_ratio, calculate_sortino_ratio, calculate_calmar_ratio

## 1. Load and Prepare Historical Bitcoin Data

First, let's fetch historical Bitcoin price data from Binance to use for our backtests.

In [ ]:
def get_historical_klines(symbol="BTCUSDT", interval="15m", start_time=None, end_time=None, limit=1000):
    """Fetch historical klines (candlestick data) from Binance API"""
    
    if end_time is None:
        end_time = datetime.now()
    if start_time is None:
        # Default to 90 days of data
        start_time = end_time - timedelta(days=90)
    
    # Convert times to milliseconds
    start_ts = int(start_time.timestamp() * 1000)
    end_ts = int(end_time.timestamp() * 1000)
    
    url = "https://api.binance.com/api/v3/klines"
    all_klines = []
    
    current_start = start_ts
    while current_start < end_ts:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': current_start,
            'endTime': end_ts,
            'limit': limit
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            klines = response.json()
            
            if not klines:
                break
                
            all_klines.extend(klines)
            current_start = int(klines[-1][0]) + 1
            
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    # Convert to dataframe
    if all_klines:
        columns = ['time', 'open', 'high', 'low', 'close', 'volume', 
                   'close_time', 'quote_asset_volume', 'number_of_trades',
                   'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
        
        df = pd.DataFrame(all_klines, columns=columns)
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        
        # Convert numeric columns
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        df[numeric_columns] = df[numeric_columns].astype(float)
        
        return df
    
    return None

# Get 3 months of hourly data
end_date = datetime.now()
start_date = end_date - timedelta(days=90)
df = get_historical_klines("BTCUSDT", "1h", start_date, end_date)

print(f"Loaded {len(df)} rows of Bitcoin price data from {df['time'].min()} to {df['time'].max()}")
df.head()

## 2. Define Simple Trading Strategies

Now let's define some simple trading strategies to test with our backtesting framework.

In [ ]:
class SimpleMovingAverageStrategy(Strategy):
    """Simple Moving Average Crossover Strategy"""
    
    def __init__(self, name="SMA Crossover", initial_balance=10000, 
                short_window=20, long_window=50, stop_loss_pct=0.05, 
                take_profit_pct=0.1, position_size=1.0):
        super().__init__(name=name, initial_balance=initial_balance)
        
        self.short_window = short_window
        self.long_window = long_window
        self.stop_loss_pct = stop_loss_pct
        self.take_profit_pct = take_profit_pct
        self.position_size = position_size
    
    def calculate_signals(self, df):
        """Calculate trading signals based on moving average crossovers"""
        # Calculate short and long moving averages
        df = df.copy()
        df['short_ma'] = df['close'].rolling(window=self.short_window).mean()
        df['long_ma'] = df['close'].rolling(window=self.long_window).mean()
        
        # Calculate signal: 1 when short_ma crosses above long_ma, -1 when short_ma crosses below long_ma
        df['signal'] = 0
        df.loc[df['short_ma'] > df['long_ma'], 'signal'] = 1
        df.loc[df['short_ma'] <= df['long_ma'], 'signal'] = -1
        
        # Calculate signal changes
        df['signal_change'] = df['signal'].diff()
        
        return df
    
    def decide_action(self, current_data):
        """
        Decide action based on moving average signals
        """
        # Skip if we don't have enough data for signals
        if 'signal_change' not in current_data:
            return Action.HOLD
        
        # Buy signal: short MA crosses above long MA
        if current_data['signal_change'] > 0:
            return Action.BUY
            
        # Sell signal: short MA crosses below long MA
        elif current_data['signal_change'] < 0 and self.position:
            return Action.SELL
            
        # Otherwise hold
        return Action.HOLD


class RSIStrategy(Strategy):
    """Simple RSI Strategy"""
    
    def __init__(self, name="RSI Strategy", initial_balance=10000, 
                rsi_period=14, oversold=30, overbought=70, 
                stop_loss_pct=0.05, take_profit_pct=0.1, position_size=1.0):
        super().__init__(name=name, initial_balance=initial_balance)
        
        self.rsi_period = rsi_period
        self.oversold = oversold
        self.overbought = overbought
        self.stop_loss_pct = stop_loss_pct
        self.take_profit_pct = take_profit_pct
        self.position_size = position_size
    
    def calculate_signals(self, df):
        """Calculate trading signals based on RSI indicator"""
        df = df.copy()
        
        # Calculate RSI
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(window=self.rsi_period).mean()
        avg_loss = loss.rolling(window=self.rsi_period).mean()
        
        rs = avg_gain / avg_loss
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # Calculate signals
        df['signal'] = 0
        df.loc[df['rsi'] < self.oversold, 'signal'] = 1  # Buy when oversold
        df.loc[df['rsi'] > self.overbought, 'signal'] = -1  # Sell when overbought
        
        # Calculate signal changes
        df['rsi_buy_signal'] = (df['rsi'] < self.oversold) & (df['rsi'].shift(1) >= self.oversold)
        df['rsi_sell_signal'] = (df['rsi'] > self.overbought) & (df['rsi'].shift(1) <= self.overbought)
        
        return df
    
    def decide_action(self, current_data):
        """
        Decide action based on RSI signals
        """
        # Skip if we don't have RSI data yet
        if 'rsi' not in current_data:
            return Action.HOLD
        
        # Buy signal: RSI crosses below oversold level
        if 'rsi_buy_signal' in current_data and current_data['rsi_buy_signal'] and not self.position:
            return Action.BUY
        
        # Sell signal: RSI crosses above overbought level
        if 'rsi_sell_signal' in current_data and current_data['rsi_sell_signal'] and self.position:
            return Action.SELL
            
        # Otherwise hold
        return Action.HOLD

## 3. Run Simple Backtest

Let's run a backtest with our simple moving average strategy.

In [ ]:
# Add technical indicators to our dataframe
df_indicators = add_all_indicators(df)

# Create a strategy instance
sma_strategy = SimpleMovingAverageStrategy(
    short_window=20,
    long_window=50,
    stop_loss_pct=0.05,
    take_profit_pct=0.10
)

# Create backtester
backtester = Backtester(sma_strategy)

# Run backtest
results = backtester.run_backtest(df_indicators)

# Print summary
print("\n----- Backtest Results -----")
print(f"Strategy: {results['strategy_name']}")
print(f"Initial Balance: ${results['initial_balance']}")
print(f"Final Balance: ${results['final_balance']:.2f}")
print(f"Total Return: {results['total_return_percent']:.2f}%")
print(f"Buy & Hold Return: {results['buy_hold_return']:.2f}%")
print(f"Outperformance: {results['outperformance']:.2f}%")
print(f"Max Drawdown: {results['max_drawdown_percent']:.2f}%")
print(f"Sharpe Ratio: {results['sharpe_ratio']:.2f}")
print(f"Win Rate: {results['win_rate_percent']:.2f}% ({results['total_trades']} trades)")
print(f"Profit Factor: {results['profit_factor']:.2f}")

# Visualize results
backtester.plot_results(results)

## 4. Strategy Optimization

Now let's optimize the parameters of our strategy to find the best combination.

In [ ]:
# Define parameter grid for optimization
param_grid = {
    'short_window': [10, 20, 30],
    'long_window': [40, 50, 60],
    'stop_loss_pct': [0.03, 0.05, 0.07],
    'take_profit_pct': [0.06, 0.09, 0.12]
}

# Optimize strategy parameters
best_params, best_result = backtester.optimize_strategy_parameters(
    df_indicators, 
    param_grid,
    metric='total_return_percent',
    maximize=True,
    n_jobs=2  # Adjust based on your CPU cores
)

# Print optimization results
print("\n----- Optimization Results -----")
print(f"Best Parameters: {best_params}")
print(f"Best Return: {best_result['total_return_percent']:.2f}%")
print(f"Best Sharpe Ratio: {best_result['sharpe_ratio']:.2f}")
print(f"Win Rate: {best_result['win_rate_percent']:.2f}%")

# Create strategy with optimized parameters
optimized_sma_strategy = SimpleMovingAverageStrategy(
    name="Optimized SMA",
    **best_params
)

# Create backtester
optimized_backtester = Backtester(optimized_sma_strategy)

# Run backtest with optimized parameters
optimized_results = optimized_backtester.run_backtest(df_indicators)

# Visualize optimized results
optimized_backtester.plot_results(optimized_results)

## 5. Walk-Forward Analysis

Let's perform walk-forward analysis to test how well our strategy performs when optimized over rolling windows of data.

In [ ]:
# Perform walk-forward analysis
wfa_results = backtester.walk_forward_analysis(
    df_indicators,
    window_size=720,  # 30 days (24 hours * 30)
    step_size=168,    # 7 days (24 hours * 7)
    param_grid=param_grid,
    metric='total_return_percent',
    maximize=True,
    verbose=True
)

# Visualize walk-forward results
print("\n----- Walk-Forward Analysis Results -----")
print(f"Number of windows: {len(wfa_results)}")
print("\nParameter stability across windows:")

# Extract best parameters for each window
param_values = {}
for window_params, _ in wfa_results:
    for param, value in window_params.items():
        if param not in param_values:
            param_values[param] = []
        param_values[param].append(value)

# Calculate and show statistics for each parameter
for param, values in param_values.items():
    print(f"{param}: mean={np.mean(values):.4f}, std={np.std(values):.4f}, min={np.min(values)}, max={np.max(values)}")

# Plot equity curves for each window
plt.figure(figsize=(12, 6))

for i, (params, result) in enumerate(wfa_results):
    plt.plot(result['portfolio_values'], label=f"Window {i+1}")

plt.title("Equity Curves Across Walk-Forward Windows")
plt.xlabel("Bar")
plt.ylabel("Portfolio Value ($)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Compare Multiple Strategies

Let's compare the performance of different trading strategies.

In [ ]:
# Create strategies to compare
strategies = [
    SimpleMovingAverageStrategy(name="SMA Strategy", short_window=20, long_window=50),
    RSIStrategy(name="RSI Strategy", rsi_period=14, oversold=30, overbought=70)
]

# Compare strategies
comparison_results = backtester.compare_strategies(strategies, df_indicators)

# Use the trading_visualization module for more advanced visualizations
for name, result in comparison_results.items():
    print(f"\n----- {name} Results -----")
    print(f"Total Return: {result['total_return_percent']:.2f}%")
    print(f"Max Drawdown: {result['max_drawdown_percent']:.2f}%")
    print(f"Sharpe Ratio: {result['sharpe_ratio']:.2f}")
    print(f"Win Rate: {result['win_rate_percent']:.2f}%")
    print(f"Total Trades: {result['total_trades']}")

    # Use advanced visualization functions
    print(f"\nGenerating charts for {name}...")
    
    # Create equity curve
    equity_fig = plot_equity_curve(result)
    plt.figure(equity_fig.number)
    plt.title(f"{name} - Equity Curve")
    plt.tight_layout()
    plt.show()
    
    # Create underwater curve (drawdown)
    underwater_fig = plot_underwater_curve(result)
    plt.figure(underwater_fig.number)
    plt.title(f"{name} - Underwater Curve")
    plt.tight_layout()
    plt.show()
    
    # Create returns distribution
    returns_fig = plot_returns_distribution(result)
    plt.figure(returns_fig.number)
    plt.title(f"{name} - Returns Distribution")
    plt.tight_layout()
    plt.show()

## 7. Conclusion

In this notebook, we've demonstrated how to use our custom backtesting framework to:

1. Run basic strategy backtests
2. Optimize strategy parameters
3. Perform walk-forward analysis
4. Compare different trading strategies
5. Visualize trading results

Key takeaways:
- Parameter optimization can significantly improve strategy performance
- Walk-forward analysis helps ensure strategies are robust over time
- Different visualization methods help understand trading performance from multiple angles
- Comparing strategies helps identify which approach works best for specific market conditions